In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import pandas as pd

In [2]:
image_dir = "/kaggle/input/flood-area-segmentation/Image"
mask_dir = "/kaggle/input/flood-area-segmentation/Mask"

images = [os.path.join(image_dir, file) for file in os.listdir(image_dir)]
masks = [os.path.join(mask_dir, file) for file in os.listdir(mask_dir)]

data = pd.DataFrame({"image": images, "mask": masks})

train_df, val_df = train_test_split(data, train_size=0.8, shuffle=True, random_state=123)

In [3]:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

def create_generator(image_generator, mask_generator, batch_size=16):
    while True:
        for (img_batch, mask_batch) in zip(image_generator, mask_generator):
            yield img_batch, mask_batch

In [4]:
train_image_generator = image_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=True,
    seed=42
)

train_mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="mask",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='grayscale',
    shuffle=True,
    seed=42
)

train_generator = create_generator(train_image_generator, train_mask_generator)

val_image_generator = image_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="image",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=False,  # Validation data should not be shuffled
    seed=42
)

val_mask_generator = mask_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="mask",
    y_col=None,
    target_size=(128, 128),
    batch_size=16,
    class_mode=None,
    color_mode='grayscale',
    shuffle=False,  # Validation data should not be shuffled
    seed=42
)

val_generator = create_generator(val_image_generator, val_mask_generator)

Found 232 validated image filenames.
Found 232 validated image filenames.
Found 58 validated image filenames.
Found 58 validated image filenames.


In [5]:
def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    
    
    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    
    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    
    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    
    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [6]:
model = unet_model(input_size=(128, 128, 3))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 8, 8, 512) │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 8, 8,      │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 8, 8,      │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │  2,097,664 │ conv2d_9[0][0]    │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 16,    │          0 │ conv2d_transpose

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    steps_per_epoch=len(train_df) // 16,
    validation_steps=len(val_df) // 16
)

Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 198s 14s/step - accuracy: 0.4692 - loss: 0.6938 - val_accuracy: 0.5839 - val_loss: 0.6803
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.6023 - loss: 0.6718 - val_accuracy: 0.5942 - val_loss: 0.6707
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.5602 - loss: 0.6861 - val_accuracy: 0.5968 - val_loss: 0.6760
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.5863 - loss: 0.6775 - val_accuracy: 0.5841 - val_loss: 0.6759
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.6012 - loss: 0.6689 - val_accuracy: 0.5839 - val_loss: 0.6770


In [13]:
loss, accuracy = model.evaluate(val_generator, steps=50)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

50/50 ━━━━━━━━━━━━━━━━━━━━ 176s 4s/step - accuracy: 0.5896 - loss: 0.6746
Validation Loss: 0.674543559551239
Validation Accuracy: 0.5894536375999451


In [ ]:
import matplotlib.pyplot as plt

def visualize_data(image_batch, mask_batch, batch_size=4):
    plt.figure(figsize=(15, 10))
    for i in range(batch_size):
        plt.subplot(2, batch_size, i + 1)
        plt.imshow(image_batch[i])
        plt.axis('off')
        plt.title('Image')
        
        plt.subplot(2, batch_size, i + 1 + batch_size)
        plt.imshow(mask_batch[i].squeeze(), cmap='gray')
        plt.axis('off')
        plt.title('Mask')
    plt.show()

image_batch, mask_batch = next(train_generator)

visualize_data(image_batch, mask_batch)

In [4]:
def visualize_predictions(model, generator):
    images, masks = next(generator)

    predictions = model.predict(images)

    for i in range(len(images)):
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 3, 1)
        plt.title("Input Image")
        plt.imshow(images[i])
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.title("True Mask")
        plt.imshow(masks[i].squeeze(), cmap='gray')
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.title("Predicted Mask")
        plt.imshow(predictions[i].squeeze(), cmap='gray')
        plt.axis('off')
        
        plt.show()

visualize_predictions(model, val_generator)

NameError: name 'model' is not defined

In [3]:
def overlay_predictions(model, generator, alpha=0.5):
    images, masks = next(generator)
    predictions = model.predict(images)

    for i in range(len(images)):
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.title("Input Image")
        plt.imshow(images[i])
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.title("Overlayed Mask")
        plt.imshow(images[i])
        plt.imshow(predictions[i].squeeze(), cmap='jet', alpha=alpha)
        plt.axis('off')
        
        plt.show()

overlay_predictions(model, val_generator)

NameError: name 'model' is not defined

In [ ]:
import cv2

def contour_overlay_predictions(model, generator):
    images, masks = next(generator)
    predictions = model.predict(images)

    for i in range(len(images)):
        plt.figure(figsize=(12, 6))
        
        binary_pred = predictions[i].squeeze() > 0.5
        
        contours, _ = cv2.findContours(binary_pred.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        image_with_contours = images[i].copy()
        cv2.drawContours(image_with_contours, contours, -1, (255, 0, 0), 2)
        
        plt.title("Contours Overlay")
        plt.imshow(image_with_contours)
        plt.axis('off')
        plt.show()

contour_overlay_predictions(model, val_generator)

In [ ]:
def heatmap_visualization(model, generator):
    images, masks = next(generator)
    predictions = model.predict(images)

    for i in range(len(images)):
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.title("Input Image")
        plt.imshow(images[i])
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.title("Prediction Heatmap")
        plt.imshow(predictions[i].squeeze(), cmap='hot')
        plt.axis('off')
        
        plt.show()

heatmap_visualization(model, val_generator)